In [92]:
import sys
import itertools
from anytree import Node, RenderTree,Resolver,ChildResolverError,NodeMixin,PreOrderIter
from anytree.dotexport import RenderTreeGraph

class CustomNode(NodeMixin):  # Add state number feature
    def __init__(self, name, state_number,symbol,end_state, parent=None):
        super(CustomNode, self).__init__()
        self.name=name
        self.state_number=state_number
        self.parent=parent
        self.end_state=end_state
        self.symbol=symbol

state_number=0
top=CustomNode("root",state_number,'',False)
r = Resolver('name')




In [93]:
def add_inflectional_suffixes(suffix,state_number,root_node):
    suffixes=suffix.split(":")
    state_number+=1
    node=CustomNode('<eps>',state_number,'-',False,root_node)
    for ch in suffixes[0][1:]:
        state_number+=1
        node=CustomNode(ch,state_number,ch,False,node)
    state_number+=1
    CustomNode('<eps>',state_number,suffixes[1],True,node)

    if suffixes[1]=='+N':
        state_number+=1
        plural_node=CustomNode('^s',state_number,'+PL',True,node)
        state_number+=1
        CustomNode("^s",state_number,'+SGPS',True,node)
        state_number+=1
        CustomNode("'",state_number,'+PLPS',True,plural_node)
    elif suffixes[1]=='+V':
        state_number+=1
        CustomNode("^s",state_number,'+3D',True,node)
        state_number+=1
        CustomNode("ing",state_number,'+ING',True,node)
    return (node,state_number)
def append_suffixes(root_node,morphs,index,state_number):
    suffix=''
    while len(morphs)>index:
        index+=1
        if morphs[index]=='|':
            if suffix!='': #check if suffix to be handle before pipe
                
                node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
                suffix=''
        elif morphs[index]=='(':
            node,state_number=add_inflectional_suffixes(suffix,state_number,root_node)
            index,state_number=append_suffixes(node,morphs,index,state_number)
            suffix=''
        elif morphs[index]==')':
            if suffix!='': #check if suffix to be handle before paranthesis
                val=add_inflectional_suffixes(suffix,state_number,root_node)
                state_number=val[1]
            return (index,state_number)
        else:
            suffix+=morphs[index]
    return (index,state_number)

def insert_irregular_noun(stem,state_number,root):
    top=root
    verbs=stems=stem.split('/')
    for i in range(len(verbs[0])):
        state_number+=1
        root=CustomNode(verbs[0][i],state_number,verbs[0][i],False,root)
    end_ptr=root
   
    
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+N',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+SG',True,end_ptr)
   
    end_ptr=top
    for i in range(len(verbs[0])):
        state_number+=1
        end_ptr=CustomNode('<eps>',state_number,verbs[0][i],False,end_ptr)
   
    
    for i in range(len(verbs[1])):
        state_number+=1
        end_ptr=CustomNode(verbs[1][i],state_number,'<eps>',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+N',False,end_ptr)
    state_number+=1
    end_ptr=CustomNode('<eps>',state_number,'+PL',True,end_ptr)
            
    return (root,state_number)

In [94]:
#words = ['compute (er:+N(ize:+V(ation:+N|able:+ADJ)|less:+ADJ(ness:+N))|ation:+N(able:+ADJ))','work (er:+N(ism:+N))']
#words = ["be/were&was/been (ing:+N)","begin/began/begun (ing:+N|er:+N)"]
with open('Lexicon/Nouns.txt') as f:
    words = f.readlines()
    words.sort()
    for word in words:
        line=word.rstrip()
        arr=line.split(' ')
        stem=arr[0]
        if len(arr)==2:
            morphs=arr[1]
        root=top

        if '/' in stem: #means irregular verb
            root,state_number=insert_irregular_noun(stem,state_number,top)
        else:
            for i in range(len(stem)):
                try:
                    n=r.get(root,stem[i])
                except ChildResolverError:
                    state_number+=1
                    n=CustomNode(stem[i],state_number,stem[i],False,root)
                root=n
            
            state_number+=1
            end_point=CustomNode("<eps>",state_number,'+N',False,root)
            state_number+=1
            CustomNode("<eps>",state_number,'+SG',True,end_point)
            state_number+=1
            CustomNode("^s",state_number,'+PL',True,end_point)
        
        if len(arr)==2:
            return_val=append_suffixes(root,morphs,0,state_number)
            state_number=return_val[1]
RenderTreeGraph(top).to_picture("udo.png")

In [95]:
p=PreOrderIter(top)

p=itertools.islice(p,1,None) #pass root


for n in p:
    print(n.name + " " +str(n.state_number) + "parent: " + n.parent.name)

a 1parent: root
c 2parent: a
t 3parent: c
<eps> 4parent: t
<eps> 5parent: <eps>
^s 6parent: <eps>
i 7parent: a
m 8parent: i
<eps> 9parent: m
<eps> 10parent: <eps>
^s 11parent: <eps>
n 12parent: a
s 13parent: n
w 14parent: s
e 15parent: w
r 16parent: e
<eps> 17parent: r
<eps> 18parent: <eps>
^s 19parent: <eps>
r 20parent: a
t 21parent: r
<eps> 22parent: t
<eps> 23parent: <eps>
^s 24parent: <eps>
<eps> 25parent: t
i 26parent: <eps>
s 27parent: i
t 28parent: s
<eps> 29parent: t
^s 30parent: t
' 32parent: ^s
^s 31parent: t
<eps> 33parent: t
r 34parent: <eps>
y 35parent: r
<eps> 36parent: y
^s 37parent: y
' 39parent: ^s
^s 38parent: y
<eps> 40parent: t
i 41parent: <eps>
c 42parent: i
<eps> 43parent: c
<eps> 44parent: c
a 45parent: <eps>
l 46parent: a
<eps> 47parent: l
<eps> 48parent: t
y 49parent: <eps>
<eps> 50parent: y
t 51parent: a
t 52parent: t
a 53parent: t
c 54parent: a
k 55parent: c
<eps> 56parent: k
<eps> 57parent: <eps>
^s 58parent: <eps>
b 59parent: root
a 60parent: b
n 61parent: 

In [96]:

#for pre, _, node in RenderTree(top):
#    treestr = u"%s%s" % (pre, node.name)
#    print(treestr.ljust(8), node.state_number)
def print_as_tabs(prev_state,next_state,input_label,output_label):
    print(str(prev_state) + "\t" + str(next_state) + "\t"+ str(input_label) + "\t" + str(output_label))


# In[222]:


p=PreOrderIter(top)
p=itertools.islice(p,1,None) #pass root

print_as_tabs(0,1,"<s>","<s>")
print_as_tabs(1,0,"<eps>","<eps>")
print_as_tabs(0,2,"</s>","</s>")
print_as_tabs(2,0,"<eps>","<eps>")
print_as_tabs(0,3,"<unk>","<unk>")
print_as_tabs(3,0,"<eps>","<eps>")
for n in p:
    parent_state_number=n.parent.state_number
    if n.parent.state_number!=0:
        parent_state_number+=3
    print_as_tabs(parent_state_number,n.state_number+3,n.symbol,n.name)
    if n.end_state:
        print_as_tabs(n.state_number+3,0,"<eps>","#")  
    #if n.is_leaf:
     #   print_as_tabs(n.state_number+3,0,"<eps>","#")
print("0")


0	1	<s>	<s>
1	0	<eps>	<eps>
0	2	</s>	</s>
2	0	<eps>	<eps>
0	3	<unk>	<unk>
3	0	<eps>	<eps>
0	4	a	a
4	5	c	c
5	6	t	t
6	7	+N	<eps>
7	8	+SG	<eps>
8	0	<eps>	#
7	9	+PL	^s
9	0	<eps>	#
4	10	i	i
10	11	m	m
11	12	+N	<eps>
12	13	+SG	<eps>
13	0	<eps>	#
12	14	+PL	^s
14	0	<eps>	#
4	15	n	n
15	16	s	s
16	17	w	w
17	18	e	e
18	19	r	r
19	20	+N	<eps>
20	21	+SG	<eps>
21	0	<eps>	#
20	22	+PL	^s
22	0	<eps>	#
4	23	r	r
23	24	t	t
24	25	+N	<eps>
25	26	+SG	<eps>
26	0	<eps>	#
25	27	+PL	^s
27	0	<eps>	#
24	28	-	<eps>
28	29	i	i
29	30	s	s
30	31	t	t
31	32	+N	<eps>
32	0	<eps>	#
31	33	+PL	^s
33	0	<eps>	#
33	35	+PLPS	'
35	0	<eps>	#
31	34	+SGPS	^s
34	0	<eps>	#
31	36	-	<eps>
36	37	r	r
37	38	y	y
38	39	+N	<eps>
39	0	<eps>	#
38	40	+PL	^s
40	0	<eps>	#
40	42	+PLPS	'
42	0	<eps>	#
38	41	+SGPS	^s
41	0	<eps>	#
31	43	-	<eps>
43	44	i	i
44	45	c	c
45	46	+ADJ	<eps>
46	0	<eps>	#
45	47	-	<eps>
47	48	a	a
48	49	l	l
49	50	+ADJ	<eps>
50	0	<eps>	#
24	51	-	<eps>
51	52	y	y
52	53	+ADJ	<eps>
53	0	<eps>	#
4	54	t	t
54	55	t	t
55	56	a	a
56	57	c	c
57	58	k	k


In [97]:
def append_suffixes(node_stack,morphs,index,paranthesis_stack,state_number):
    suffix=''
    while len(paranthesis_stack)!=0:
        index+=1
        
        if morphs[index]=='|':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
                suffix=''
        elif morphs[index]=='(':
            suffixes=suffix.split(':')
            state_number+=1
            
            node=CustomNode(suffixes[0],state_number,suffixes[1],node_stack[-1])
            node_stack.append(node)
            paranthesis_stack.append(index)
            param_node_stack=[node_stack[-1]]
            param_paranthesis_stack=[index]
            print(node_stack[-1].name)
            print(param_paranthesis_stack)
        
            var=append_suffixes(param_node_stack,morphs,index,param_paranthesis_stack,state_number)
            index=var[0]
            state_number=var[1]
            node_stack.pop()
            paranthesis_stack.pop()
            suffix=''
            print(node_stack[-1].name)
        elif morphs[index]==')':
            if suffix!='':
                node=node_stack[-1] 
                state_number+=1
                suffixes=suffix.split(':')
                CustomNode(suffixes[0],state_number,suffixes[1],node)
            paranthesis_stack.pop()
            node_stack.pop()
            return (index,state_number,node_stack,paranthesis_stack)
        else:
            suffix+=morphs[index]
    return (index,state_number,node_stack,paranthesis_stack)